####Ingestion from source datalake to bronze layer

In [0]:
dbutils.widgets.text("catalog","")
CATALOG = dbutils.widgets.get("catalog").strip()
dbutils.widgets.text("schema","")
SCHEMA = dbutils.widgets.get("schema").strip()

#####Setting generic configuration

In [0]:
import json # since json is used in both child (json.dumps()) and parent (json.load()) notrbooks it is required to import json in both notebooks.

# when we run the child notebook, thru exit() we will receive the string values to make a dict. we have to parse by calling it as json.loads().

config_nb_out = dbutils.notebook.run(
    "/Workspace/Users/sangeetha.va@outlook.com/databricks-code-repo/databricks_workouts_2025/4_logistics_usecase/Medallion Arch/generic_medallionArch_Practice/General_conf_utils/configs_path",120,{"catalog":CATALOG,"schema":SCHEMA})

#storing the output from child notebook to a parent notebook variable.
config_dict = json.loads(config_nb_out)
print(config_dict)

#storing src & bronze file location to variable in parent notebook
SRC = config_dict["SRC"]
BRONZE = config_dict["BRONZE"]

print("source location", SRC)
print("bronze location",BRONZE)


#####Defining functions

In [0]:
%run "/Workspace/Users/sangeetha.va@outlook.com/databricks-code-repo/databricks_workouts_2025/4_logistics_usecase/Medallion Arch/generic_medallionArch_Practice/General_conf_utils/util_functions"

#####All Generic Functions can be defined here like spark session initiation/read opn/write opn etc.

In [0]:
spark=get_spark_session("logistics medallion arch-FW practice")

#from pyspark.sql.sessions import SparkSession
# in general platform would have spark session instatiated automatically, yet if no active session then config and create one
'''def get_spark_session(app_name="Medallion-Arch Practice"):
    try:
        spark = SparkSession.getActiveSession() #calling active session if any & rtn
        if spark:
            return spark
    except:
        pass 
    return (SparkSession.builder.config ("spark.sql.shuffle.partitions", "1").appName(app_name).getOrCreate())'''
    # here builder is entry point for config the current spark session. further spark sql config is done for 1 partition since its small vol. of data, by default it would use 200. finally get or create a session.

'''We lost all the below features if using inline functions...
    Centralized and controllable
    Production Ready
    Reusability
    Seperation of Concern
    Modularized
    Simple to write/Reasonable to understand
    Optimization
    Governed
    Secured
'''


In [0]:
# read all files and tables

staff1 = read_file(spark,'csv',f"{SRC}/logistics_source1.txt",True,False)# using function in if/elif section
staff2 = read_csv_df(spark,f"{SRC}/logistics_source2.txt",True,False)# using function defined seperately in util. notebook
print(staff1)
print(staff2)

staff_bronze = mergeDf(staff1,staff2,True) # since src1 & 2 has different structure we are using merge opn.

#geotag file read
geo_tagging=read_csv_df(spark,f"{SRC}/Master_City_List.csv",True,False)

#Shipment data read
shipments_bronze = read_json_df(spark,f"{SRC}/logistics_shipment_detail_3000.json",True)

In [0]:
print("source location", SRC)
print("bronze location", BRONZE)

#write_file(staff1, f"{BRONZE}/staff", mode="overwrite", format="delta") 
'''- to debug issue, tried to write staff1 to bronze. later when issue was resolved, I wanted to write merged stage_bronze to bronze target which has differnt schema.Hence to overwrite schema without deleting the existing file I can use option("overwriteSchema","True"). this works only when mode="overwrite".'''

write_file(staff_bronze, f"{BRONZE}/staff", mode="overwrite", format="delta")
write_file(geo_tagging, f"{BRONZE}/geotag", mode="overwrite", format="delta")
write_file(shipments_bronze, f"{BRONZE}/shipments", mode="overwrite", format="delta")